In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.2 MB/s 
     |████████████████████████████████| 6.6 MB 39.3 MB/s 
     |████████████████████████████████| 101 kB 10.5 MB/s 
     |████████████████████████████████| 596 kB 78.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.1 MB/s 
     |████████████████████████████████| 6.4 MB 31.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('/content/drive/MyDrive/JU Project/ExecutiveTweetsAAPL.csv')
df

,Date,Tweet
0,2017-06-25 16:41:15+00:00,Poll: One week after Apple announces it is acq...
1,2020-08-07 18:41:26+00:00,If only Apple owned $100B worth of Berkshire i...
2,2019-11-01 17:36:36+00:00,Berkshire Hathaway's stake in Apple now repres...
3,2019-12-30 15:37:15+00:00,The #IBDLive Team running through the stock re...
4,2020-10-30 15:39:01+00:00,75% of Apple Watch sales in the quarter were t...
...,...,...
255,2020-07-30 21:37:48+00:00,Dear @federalreserve AAPL results demonstrated...
256,2020-01-27 17:06:30+00:00,"Goldman on AAPL: ""we have begun to detect incr..."
257,2019-11-07 18:18:11+00:00,APPLE SAYS IT WILL ISSUE AN ADDED $2.2B IN GRE...
258,2019-03-20 12:42:47+00:00,Apple announces AirPods with Apple-designed H1...


In [6]:
df['Date'] = pd.to_datetime(df['Date']).dt.date
df

,Date,Tweet
0,2017-06-25,Poll: One week after Apple announces it is acq...
1,2020-08-07,If only Apple owned $100B worth of Berkshire i...
2,2019-11-01,Berkshire Hathaway's stake in Apple now repres...
3,2019-12-30,The #IBDLive Team running through the stock re...
4,2020-10-30,75% of Apple Watch sales in the quarter were t...
...,...,...
255,2020-07-30,Dear @federalreserve AAPL results demonstrated...
256,2020-01-27,"Goldman on AAPL: ""we have begun to detect incr..."
257,2019-11-07,APPLE SAYS IT WILL ISSUE AN ADDED $2.2B IN GRE...
258,2019-03-20,Apple announces AirPods with Apple-designed H1...


In [7]:
def tweets_per_day(df):
    df['Date'] = pd.to_datetime(df['Date'])
#   return df[['Tweets']].groupby(df['Date'].dt.date).count()
    tweet = df['Tweet'].groupby(df['Date'].dt.date).count() 
    d1 = pd.DataFrame(tweet)
    return d1
  # if you want output to be `Series`

df1 = tweets_per_day(df)
df1 = df1.reset_index(level=0)

In [8]:
df1

,Date,Tweet
0,2017-01-03,1
1,2017-01-11,1
2,2017-01-17,1
3,2017-02-01,2
4,2017-02-07,1
...,...,...
191,2022-04-25,1
192,2022-04-28,1
193,2022-05-11,1
194,2022-05-12,1


In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

In [10]:
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
import math

stop_words = stopwords.words('english')
stop_words.remove("not")
stemmer = SnowballStemmer('english')

text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

def preprocess(text, stem=False):
  text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
  tokens = []
  for token in text.split():
    if token not in stop_words:
      if stem:
        tokens.append(stemmer.stem(token))
      else:
        tokens.append(token)
  return " ".join(tokens)

df.Tweets = df.Tweet.apply(lambda x: preprocess(x))
df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


,Date,Tweet
0,2017-06-25,Poll: One week after Apple announces it is acq...
1,2020-08-07,If only Apple owned $100B worth of Berkshire i...
2,2019-11-01,Berkshire Hathaway's stake in Apple now repres...
3,2019-12-30,The #IBDLive Team running through the stock re...
4,2020-10-30,75% of Apple Watch sales in the quarter were t...
...,...,...
255,2020-07-30,Dear @federalreserve AAPL results demonstrated...
256,2020-01-27,"Goldman on AAPL: ""we have begun to detect incr..."
257,2019-11-07,APPLE SAYS IT WILL ISSUE AN ADDED $2.2B IN GRE...
258,2019-03-20,Apple announces AirPods with Apple-designed H1...


In [11]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [12]:
neg = []
neu = []
pos = []
i = 0
def normalize(score, alpha=15):
    """
    Normalize the score to be between -1 and 1 using an alpha that
    approximates the max expected value
    """
    norm_score = score/math.sqrt((score*score) + alpha)
    return norm_score

for tweet in df.Tweet:
  i = i+1
  encoded_tweet = tokenizer(tweet,padding=True, truncation=True,max_length=250, add_special_tokens = True, return_tensors='pt')
  output = model(**encoded_tweet)
  scores = output[0][0].detach().numpy()
  normScores = softmax(scores)  
  neg.append(normScores[0])
  neu.append(normScores[1])
  pos.append(normScores[2])
  print(i,"--> ",normScores)

1 -->  [0.0286408  0.03915861 0.93220073]
2 -->  [0.05777333 0.02842432 0.9138022 ]
3 -->  [0.8581456  0.01033527 0.13151924]
4 -->  [0.01051808 0.9687463  0.02073549]
5 -->  [0.38558993 0.01519462 0.5992155 ]
6 -->  [0.11288286 0.03015336 0.8569638 ]
7 -->  [0.09733891 0.01331197 0.88934904]
8 -->  [0.7282498  0.00831909 0.26343116]
9 -->  [0.05007716 0.0164777  0.93344504]
10 -->  [0.8215281  0.01851193 0.15996005]
11 -->  [0.41820115 0.00791922 0.5738797 ]
12 -->  [0.02352804 0.4631364  0.5133356 ]
13 -->  [0.82874274 0.00665728 0.1645999 ]
14 -->  [0.86609817 0.02892973 0.10497215]
15 -->  [0.19928378 0.70219034 0.09852584]
16 -->  [0.08218957 0.04455424 0.8732562 ]
17 -->  [0.01073613 0.9671838  0.02208018]
18 -->  [0.0174093  0.8529545  0.12963621]
19 -->  [0.03844857 0.84282917 0.11872219]
20 -->  [0.03308483 0.03387611 0.93303895]
21 -->  [0.95405895 0.02283277 0.02310831]
22 -->  [0.28201485 0.03381518 0.68416995]
23 -->  [0.90635896 0.02228806 0.07135299]
24 -->  [0.04740053 

In [13]:
df['negative'] = neg
df['neutral'] = neu
df['positive'] = pos
df

,Date,Tweet,negative,neutral,positive
0,2017-06-25,Poll: One week after Apple announces it is acq...,0.028641,0.039159,0.932201
1,2020-08-07,If only Apple owned $100B worth of Berkshire i...,0.057773,0.028424,0.913802
2,2019-11-01,Berkshire Hathaway's stake in Apple now repres...,0.858146,0.010335,0.131519
3,2019-12-30,The #IBDLive Team running through the stock re...,0.010518,0.968746,0.020735
4,2020-10-30,75% of Apple Watch sales in the quarter were t...,0.385590,0.015195,0.599216
...,...,...,...,...,...
255,2020-07-30,Dear @federalreserve AAPL results demonstrated...,0.887037,0.026398,0.086565
256,2020-01-27,"Goldman on AAPL: ""we have begun to detect incr...",0.139383,0.826010,0.034607
257,2019-11-07,APPLE SAYS IT WILL ISSUE AN ADDED $2.2B IN GRE...,0.398377,0.007877,0.593746
258,2019-03-20,Apple announces AirPods with Apple-designed H1...,0.114071,0.012669,0.873261


In [15]:
dfx = pd.DataFrame()
po = []
nu = []
ne = []
for i in range (0,196):

  d = df['Date'][i]

  x = df[df['Date'] == d]
  po.append(x['positive'].mean())
  nu.append(x['neutral'].mean())
  ne.append(x['negative'].mean())
dfx['Date'] = df1['Date']
dfx['positive'] = po
dfx['negative'] = ne
dfx['neutral'] = nu
dfx

,Date,positive,negative,neutral
0,2017-01-03,0.932201,0.028641,0.039159
1,2017-01-11,0.913802,0.057773,0.028424
2,2017-01-17,0.520478,0.467910,0.011611
3,2017-02-01,0.142369,0.362007,0.495624
4,2017-02-07,0.599216,0.385590,0.015195
...,...,...,...,...
191,2022-04-25,0.721529,0.074456,0.204015
192,2022-04-28,0.181715,0.807864,0.010421
193,2022-05-11,0.384678,0.602819,0.012503
194,2022-05-12,0.930688,0.036468,0.032844


In [16]:
# Set date as Index
dfx = dfx.set_index('Date')
dfx = dfx.sort_index()
dfx

,positive,negative,neutral
Date,,,
2017-01-03,0.932201,0.028641,0.039159
2017-01-11,0.913802,0.057773,0.028424
2017-01-17,0.520478,0.467910,0.011611
2017-02-01,0.142369,0.362007,0.495624
2017-02-07,0.599216,0.385590,0.015195
...,...,...,...
2022-04-25,0.721529,0.074456,0.204015
2022-04-28,0.181715,0.807864,0.010421
2022-05-11,0.384678,0.602819,0.012503


### STOCK DATA

In [17]:
from pandas_datareader import data as pdr
import yfinance as yf

yf.pdr_override()

stock_price = pdr.get_data_yahoo("AAPL", start = '2017-01-01', end = '2022-05-07')
print(len(stock_price))
stock_price

[*********************100%***********************]  1 of 1 completed
1346


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-03,28.950001,29.082500,28.690001,29.037500,27.257645,115127600
2017-01-04,28.962500,29.127501,28.937500,29.004999,27.227133,84472400
2017-01-05,28.980000,29.215000,28.952499,29.152500,27.365595,88774400
2017-01-06,29.195000,29.540001,29.117500,29.477501,27.670677,127007600
2017-01-09,29.487499,29.857500,29.485001,29.747499,27.924122,134247600
...,...,...,...,...,...,...
2022-05-02,156.710007,158.229996,153.270004,157.960007,157.728256,123055300
2022-05-03,158.149994,160.710007,156.320007,159.479996,159.246017,88966500
2022-05-04,159.669998,166.479996,159.259995,166.020004,165.776428,108256500


In [18]:
stock_price = stock_price.drop_duplicates()

# dropping the zero values from volume
stock_price = stock_price[stock_price['Volume'] != 0]

# filtering the important columns required
stock_price = stock_price.filter(['Date', 'Close', 'Open', 'High', 'Low', 'Volume'])

# sorting the data according to the index i.e 'Date'
stock_price = stock_price.sort_index(ascending=True, axis=0)
stock_price

,Close,Open,High,Low,Volume
Date,,,,,
2017-01-03,29.037500,28.950001,29.082500,28.690001,115127600
2017-01-04,29.004999,28.962500,29.127501,28.937500,84472400
2017-01-05,29.152500,28.980000,29.215000,28.952499,88774400
2017-01-06,29.477501,29.195000,29.540001,29.117500,127007600
2017-01-09,29.747499,29.487499,29.857500,29.485001,134247600
...,...,...,...,...,...
2022-05-02,157.960007,156.710007,158.229996,153.270004,123055300
2022-05-03,159.479996,158.149994,160.710007,156.320007,88966500
2022-05-04,166.020004,159.669998,166.479996,159.259995,108256500


### COMBINE DATA

In [19]:
stock_data = pd.merge(stock_price,dfx, how='left', left_index=True, right_index=True)

stock_data

,Close,Open,High,Low,Volume,positive,negative,neutral
Date,,,,,,,,
2017-01-03,29.037500,28.950001,29.082500,28.690001,115127600,0.932201,0.028641,0.039159
2017-01-04,29.004999,28.962500,29.127501,28.937500,84472400,NaN,NaN,NaN
2017-01-05,29.152500,28.980000,29.215000,28.952499,88774400,NaN,NaN,NaN
2017-01-06,29.477501,29.195000,29.540001,29.117500,127007600,NaN,NaN,NaN
2017-01-09,29.747499,29.487499,29.857500,29.485001,134247600,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-05-02,157.960007,156.710007,158.229996,153.270004,123055300,NaN,NaN,NaN
2022-05-03,159.479996,158.149994,160.710007,156.320007,88966500,NaN,NaN,NaN
2022-05-04,166.020004,159.669998,166.479996,159.259995,108256500,NaN,NaN,NaN


In [20]:
stock_data.sort_index(axis = 1)
stock_data

,Close,Open,High,Low,Volume,positive,negative,neutral
Date,,,,,,,,
2017-01-03,29.037500,28.950001,29.082500,28.690001,115127600,0.932201,0.028641,0.039159
2017-01-04,29.004999,28.962500,29.127501,28.937500,84472400,NaN,NaN,NaN
2017-01-05,29.152500,28.980000,29.215000,28.952499,88774400,NaN,NaN,NaN
2017-01-06,29.477501,29.195000,29.540001,29.117500,127007600,NaN,NaN,NaN
2017-01-09,29.747499,29.487499,29.857500,29.485001,134247600,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-05-02,157.960007,156.710007,158.229996,153.270004,123055300,NaN,NaN,NaN
2022-05-03,159.479996,158.149994,160.710007,156.320007,88966500,NaN,NaN,NaN
2022-05-04,166.020004,159.669998,166.479996,159.259995,108256500,NaN,NaN,NaN


In [21]:
stock_data.isna().any()

Close       False
Open        False
High        False
Low         False
Volume      False
positive     True
negative     True
neutral      True
dtype: bool

In [22]:
# rearranging the columns of the whole stock_data
stock_data = stock_data[['Close', 'negative', 'neutral', 'positive', 'Open', 'High', 'Low', 'Volume']]

# displaying the final stock_data
stock_data

,Close,negative,neutral,positive,Open,High,Low,Volume
Date,,,,,,,,
2017-01-03,29.037500,0.028641,0.039159,0.932201,28.950001,29.082500,28.690001,115127600
2017-01-04,29.004999,NaN,NaN,NaN,28.962500,29.127501,28.937500,84472400
2017-01-05,29.152500,NaN,NaN,NaN,28.980000,29.215000,28.952499,88774400
2017-01-06,29.477501,NaN,NaN,NaN,29.195000,29.540001,29.117500,127007600
2017-01-09,29.747499,NaN,NaN,NaN,29.487499,29.857500,29.485001,134247600
...,...,...,...,...,...,...,...,...
2022-05-02,157.960007,NaN,NaN,NaN,156.710007,158.229996,153.270004,123055300
2022-05-03,159.479996,NaN,NaN,NaN,158.149994,160.710007,156.320007,88966500
2022-05-04,166.020004,NaN,NaN,NaN,159.669998,166.479996,159.259995,108256500


In [23]:
stock_data.to_csv('/content/drive/MyDrive/JU Project/EXEC_FINBERT_AAPL_data.csv')